# Will It Blend? Deploying BlenderBot

First, load the model.

In [11]:


from parlai.core.agents import create_agent_from_model_file
blender_agent = create_agent_from_model_file("blender_distilled/blender_distilled.pth.checkpoint",
                                            opt_overrides={
                                                "model": "internal:cgen",
                                                "skip_generation": False,
                                                "inference": "delayedbeam",
                                                "beam_delay": 25
                                            })



03:55:46 | Overriding opt["model"] to internal:cgen (previously: projects.anti_scaling.distillation:DistillTransformerAgent)
03:55:46 | Overriding opt["skip_generation"] to False (previously: True)
03:55:46 | Overriding opt["inference"] to delayedbeam (previously: beam)
03:55:46 | Overriding opt["beam_delay"] to 25 (previously: 30)


INFO:parlai_internal.agents.cgen.cgen:Initializing a Conditional GENeration (CGEN) Agent (like transformer/generator).


03:55:46 | Using CUDA
03:55:46 | loading dictionary from blender_distilled/blender_distilled.pth.checkpoint.dict
03:55:46 | num words = 8008
03:56:03 | Total parameters: 1,122,675,200 (1,122,019,840 trainable)
03:56:03 | Loading existing model params from blender_distilled/blender_distilled.pth.checkpoint


In [13]:
%load_ext autoreload
%autoreload 2

import logging
logging.basicConfig(level=logging.DEBUG)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
blender_agent.opt["temperature"] = 0.7
blender_agent.opt["topp"] = 0.9
blender_agent.opt["topk"] = 5
blender_agent.opt["beam_min_length"] = 15
blender_agent.opt["beam_size"] = 10
blender_agent.opt["inference"] = "beam"

In [31]:
blender_agent.opt["beam_context_block_ngram"]

3

## Example

Minimal example from Github issue

In [38]:

from itertools import zip_longest
import time
from parlai.utils.strings import normalize_reply

start = time.time()

blender_agent.reset()

history = [
    "",
    "Hi, this is an Alexa Prize Socialbot. I'd like to get to know you a bit better before we chat! Is it all right if I ask for you name?",
    "trenton",
    "Nice to meet you, trenton! I'm looking forward to chatting with you today. So, I think the key to a good mood is eating well. What's a food that always makes you feel good?"
    "hazelnut",
    "I love hazelnut as well! It's nice and crunchy and I especially love it as a topping on my frozen yogurt. What do you like about hazelnut?",
    "um i guess it tastes good"
]
prefix = "I love hazelnut! It's nice as"
def process(history):
    pairs = zip_longest(*[iter(history)]*2)

    for user, bot in pairs:
        blender_agent.observe({'text': user, 'episode_done': False})
        print("Added user utterance:", user)
        if bot:
            blender_agent.history.add_reply(bot)
            print("Added bot utterance:", bot)
    response = blender_agent.act(prefix_text=prefix)
    elapsed = time.time() - start
    print(f"BlenderBot replied in {elapsed * 1000:.0f}ms: {normalize_reply(response['text'])}")
    print("All beams:", "\n".join([t for t, s in response["beam_texts"]]))
    print()

    print("BlenderBot's history view:")
    print(blender_agent.history)
    return response
    
response = process(history)

Added user utterance: 
Added bot utterance: Hi, this is an Alexa Prize Socialbot. I'd like to get to know you a bit better before we chat! Is it all right if I ask for you name?
Added user utterance: trenton
Added bot utterance: Nice to meet you, trenton! I'm looking forward to chatting with you today. So, I think the key to a good mood is eating well. What's a food that always makes you feel good?hazelnut
Added user utterance: I love hazelnut as well! It's nice and crunchy and I especially love it as a topping on my frozen yogurt. What do you like about hazelnut?
Added bot utterance: um i guess it tastes good
BlenderBot replied in 703ms: I love hazelnut! It's nice as it's one of the most widely consumed foods in the world. What about you?
All beams: I love hazelnut! It's nice as it's one of the most widely consumed foods in the world. What about you?
I love hazelnut! It's nice as it's one of the most widely consumed foods in the world!
I love hazelnut! It's nice as it's one of the mos

In [36]:
response

{'id': 'Cgen',
 'episode_done': False,
 'text': "I love hazelnut! It's nice as it's one of the most widely consumed foods in the world. What about you?",
 'beam_texts': [("I love hazelnut! It's nice as it's one of the most widely consumed foods in the world. What about you?",
   -11.454596519470215),
  ("I love hazelnut! It's nice as it's one of the most widely consumed foods in the world!",
   -11.625725746154785),
  ("I love hazelnut! It's nice as it's one of the most widely consumed fruits in the world!",
   -11.740839958190918),
  ("I love hazelnut! It's nice as it's one of the most widely consumed foods in the world.",
   -11.787345886230469),
  ("I love hazelnut! It's nice as it's one of the most widely consumed fruits in the world.",
   -11.881574630737305),
  ("I love hazelnut! It's nice as it's one of the most widely consumed foods in the world. Do you like it?",
   -11.947830200195312),
  ("I love hazelnut! It's nice as it's one of the most popular foods in the world!",
   -1

In [18]:
from itertools import zip_longest
history = [
    "hi, where are you from",
    "i'm from the states! i'm from california originally but i live in utah now.",
    "cool, i am also from utah",
    "that's great, what is your favorite place?",
    "um maybe taiwan?"
]
pairs = zip_longest(*[iter(history)]*2)
print(list(pairs))

[('hi, where are you from', "i'm from the states! i'm from california originally but i live in utah now."), ('cool, i am also from utah', "that's great, what is your favorite place?"), ('um maybe taiwan?', None)]


In [14]:
from parlai.agents.transformer.transformer import TransformerGeneratorAgent
